In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("./keys.env")

openai_api_key = os.getenv("OPEN_AI_KEY")
openai_api_key[:5]
# os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key, model_name='gpt-4')


from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema, PydanticOutputParser

system_message = SystemMessagePromptTemplate.from_template("You are a helpful assistant that, when given a video transcript, identifies the claims made and any sources provided")
human_message = HumanMessagePromptTemplate.from_template("""
{transcript}

{format_instructions}
""")


In [ ]:
from typing import List
from pydantic import BaseModel, Field, validator

class Claim(BaseModel):
    claim: str = Field(description="an exact quote from the transcript in which the claim is made")
    source: str = Field(description="the source provided for the claim or the word none if none are given")

    # @validator("claim")
    # def claim_in_transcript(cls, field):
    #     if str(field) not in transcript:
    #         raise ValueError("No bro :(")
    #     return field

class List_Claims(BaseModel):
    pairs: List[Claim]

output_parser = PydanticOutputParser(pydantic_object=List_Claims)
format_instructions = output_parser.get_format_instructions()

prompt = ChatPromptTemplate(
    messages=[
        system_message,
        human_message 
    ],
    input_variables=["transcript"],
    partial_variables={"format_instructions": format_instructions}
)

with open('test/truth/scishow_dna/transcript.txt', 'r') as f:
  transcript = f.read()[:658]

In [ ]:
# final_prompt = prompt.format(transcript=transcript)
# output = llm(final_prompt)
_input = prompt.format_prompt(transcript=transcript)
output = llm(_input.to_messages())

In [ ]:
print(output_parser.parse(output.content))
# print(output)